In [1]:
import cv2
import numpy as np
import os

import ipywidgets as widgets
from IPython.display import display, clear_output
from functools import partial
from collections import defaultdict
from ultralytics import YOLO
import time

from tiki.mini import TikiMini
from testing import TestCapture, read_realtime, capture_generator
os.environ['YOLO_VERBOSE'] = '0'

In [2]:
tiki = TikiMini()
tiki.stop()

In [3]:
# Hyper Parameters
# 실전용
#lower_event = np.array([67, 23, 101])
#upper_event = np.array([80, 255, 168])

# 연습용
lower_event = np.array([41, 57, 50])
upper_event = np.array([52, 153, 161])

#Byung Ddu ggung
#lower_event = np.array([90, 100, 100])
#upper_event = np.array([150, 255, 255])

#절연테이프
#lower_line = np.array([0, 0, 47])
#upper_line = np.array([179,147,110])

#연습용, 실전용!!!
lower_line = np.array([0, 0, 0])
upper_line = np.array([220, 70, 130])

#실전용마킹!!!
#lower_event = np.array([47, 23, 141])
#upper_event = np.array([80, 255, 168])


roix_s = 280
roix_e = 350
roiy_s = 170
roiy_e = 490

In [4]:
def stop():
    tiki.stop()

def turn_90(direction):
    turn_power1 = 15
    turn_power2 = 15

    if direction == 'left':
        tiki.counter_clockwise(turn_power1)
    elif direction == 'right':
        tiki.clockwise(turn_power2)

In [5]:
def detect_lines(image):
    
    crop_img = image[roix_s:roix_e, roiy_s:roiy_e].copy()
    
    hsv_image = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
    color_mask = cv2.inRange(hsv_image, lower_line, upper_line)
    masked_image = cv2.bitwise_and(crop_img, crop_img, mask=color_mask)
    gray_mask = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_mask, 1, 255, cv2.THRESH_BINARY)
    lines, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return crop_img, lines

In [6]:
a = 48
def adjust_wheel_speed(cx, start_cx):
    if (cx > start_cx + 85):
        tiki.set_motor_power(tiki.MOTOR_LEFT, 50)#1시 테스트때 50이엇음
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 10)
    elif (cx < start_cx - 85):
        tiki.set_motor_power(tiki.MOTOR_LEFT, 10)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 50)
    
    # MICRO CONTROL
    elif (cx > start_cx + 30):
        tiki.set_motor_power(tiki.MOTOR_LEFT, a)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 30)
    elif (cx < start_cx - 30):
        tiki.set_motor_power(tiki.MOTOR_LEFT, 30)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, a)
        
    # NORMAL
    elif (start_cx - 30 < cx < start_cx + 30): 
        tiki.set_motor_power(tiki.MOTOR_LEFT, 30)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 30) 

    # CURVE
    '''
    if (cx > start_cx + 70):
        tiki.set_motor_power(tiki.MOTOR_LEFT, 40)#1시 테스트때 50이엇음
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 10)
    elif (cx < start_cx - 70):
        tiki.set_motor_power(tiki.MOTOR_LEFT, 10)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 40)
    
    # MICRO CONTROL
    elif (cx > start_cx + 30):
        tiki.set_motor_power(tiki.MOTOR_LEFT, a)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 30)
    elif (cx < start_cx - 30):
        tiki.set_motor_power(tiki.MOTOR_LEFT, 30)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, a)
        
    # NORMAL
    elif (start_cx - 30 < cx < start_cx + 30): 
        tiki.set_motor_power(tiki.MOTOR_LEFT, 30)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 30) 
        '''
    return

In [7]:
def detect_event_color(image, lower_event, upper_event):
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_event, upper_event)
    return mask

def calculate_center_area(image, center_width=200, center_height=50):
    height, width = image.shape[:2]
    
    center_x = width // 2 
    center_y = height // 2 + 10
    start_x = center_x - center_width // 2
    start_y = center_y - center_height // 2
    end_x = start_x + center_width
    end_y = start_y + center_height

    return (start_x, start_y, end_x, end_y)

def detect_event_in_center(center_mask, start_x, start_y):
    event_detected = np.sum(center_mask) > 0
    
    event_point = None
    if event_detected:
        contours, _ = cv2.findContours(center_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"]) + start_x
                cy = int(M["m01"] / M["m00"]) + start_y
                event_point = (cx, cy)
    
    return event_detected, event_point

In [8]:
def add_callback(callbacks,callback, end_time, *argu):
    callback = partial(callback, *argu)
    callbacks.append((callback, time.time() + end_time))

In [9]:
def Event_Handler(callbacks, event_num):
    Time90 = 1.5
    if event_num == 0:
        add_callback(callbacks, turn_90, Time90, 'left')
        add_callback(callbacks, tiki.stop, 2) 
        return 20
    elif event_num == 1: # mission
        add_callback(callbacks, tiki.forward, 0.8, 30)
        add_callback(callbacks, turn_90, 2.3, 'left')
        add_callback(callbacks, tiki.stop, 2.6)  
        return 15
    elif event_num == 2:
        add_callback(callbacks, turn_90, Time90, 'left')
        add_callback(callbacks, tiki.stop, 2) 
        return 40
    elif event_num == 3: # mission
        add_callback(callbacks, tiki.forward, 0.8, 30)
        add_callback(callbacks, turn_90, 2.3, 'left')
        add_callback(callbacks, tiki.stop, 3.6)  
        return 40
    elif event_num == 4:
        add_callback(callbacks, turn_90, Time90, 'right')
        add_callback(callbacks, tiki.stop, 2) 
        return 20
    elif event_num == 5: # mission
        add_callback(callbacks, tiki.forward, 0.8, 30)
        add_callback(callbacks, turn_90, 2.3, 'left')
        add_callback(callbacks, tiki.stop, 2.6)  
        return 80
    elif event_num == 6: # mission
        add_callback(callbacks, tiki.forward, 0.8, 30)
        add_callback(callbacks, turn_90, 2.3, 'left')
        add_callback(callbacks, tiki.stop, 2.6)  
        return 20
    elif event_num == 7:
        add_callback(callbacks, turn_90, Time90, 'left')
        add_callback(callbacks, tiki.forward,3, 30)
        return 10
    elif event_num == 8:
        add_callback(callbacks, tiki.forward,2, 30)
        add_callback(callbacks, tiki.stop, 3) 
        stop()
    

In [10]:
class Predictor:
    def __init__(self, ckpt):
        self.model = YOLO(ckpt, verbose=False)
        for _ in range(5):
            results = self.model.predict('1000.jpg')[0]

    def predict(
        self,
        cap, 
        vid_widget,
        n=15,
    ):
        
        votes = defaultdict(int)
        def count_cls(preds):
            return tuple(preds.count(i) for i in range(4))
        time.sleep(0.5)
        
        for _ in range(n):
            frame = read_realtime(cap)
        
            results = self.model.predict(frame)[0]
            annot_frame = results.plot()
        
            clss = tuple(map(int, results.boxes.cls.cpu().tolist()))
            cls_cnt = count_cls(clss)
            votes[cls_cnt] += 1
            
            vid_widget.value = cv2.imencode('.bmp', annot_frame)[1].tobytes()
    
    
        print(*votes.items(), sep='\n')
    
        ret = max(votes.keys(), key=lambda x: votes[x])
        if ret[2] == 0 and ret[3]:
            #tiki.log('FIRE')
            for _ in range(3):
                tiki.fire_cannon()
                time.sleep(0.01)
        
        return ret

predictor = Predictor('branch1/pts/tank_v2.pt')
print('done')


image 1/1 /home/jetson/Workspace/1000.jpg: 480x640 1 friendly person, 1 enemy person, 1 enemy tank, 132.1ms
Speed: 6.7ms preprocess, 132.1ms inference, 7.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/jetson/Workspace/1000.jpg: 480x640 1 friendly person, 1 enemy person, 1 enemy tank, 128.3ms
Speed: 4.2ms preprocess, 128.3ms inference, 10.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/jetson/Workspace/1000.jpg: 480x640 1 friendly person, 1 enemy person, 1 enemy tank, 123.2ms
Speed: 5.3ms preprocess, 123.2ms inference, 6.7ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/jetson/Workspace/1000.jpg: 480x640 1 friendly person, 1 enemy person, 1 enemy tank, 124.7ms
Speed: 4.3ms preprocess, 124.7ms inference, 13.4ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/jetson/Workspace/1000.jpg: 480x640 1 friendly person, 1 enemy person, 1 enemy tank, 122.3ms
Speed: 4.3ms preprocess, 122.3ms inference, 10.6ms postproces

In [12]:
def main():
    video_widget = widgets.Image(format='jpeg')
    
    tiki.log_clear()
    for led_num in range(8):
        tiki.set_led_color(led_num, 0, 0, 0)
    tiki.set_motor_mode(tiki.MOTOR_MODE_PID)
    
    event_detection_count = 0
    event_detection_threshold = 3
    
    event_cnt = 0
    
    cool_flag = 1
    cool_time = 0
    cx = 0
    start_cx = None
    start_time = time.time()  # 시작 시간 기록

    callbacks = [] 
    add_callback(callbacks, tiki.stop, 8)
    Mission_idx = [1,3,5,6]
    ev_flag = False
    
    clear_output(wait=True)
    display(video_widget)

    # for frame in capture_generator():
    # with TestCapture() as cap:
    with TestCapture() as (cap, out):
        while True:
            frame = read_realtime(cap)
            if frame is None:
                break
                
            t_cur = time.time()
            for callback, end_time in callbacks:
                if t_cur <= end_time:
                    callback()
                    break
            callbacks = list(filter(lambda c_t: c_t[1] > t_cur, callbacks))
            """
            if out is not None:
                out.write(frame)
            """
            # Line
            crop_img, lines = detect_lines(frame)
            
            if len(lines) > 0:
                c = max(lines, key=cv2.contourArea)
                M = cv2.moments(c)
                if M['m00'] != 0:
                    cx = int(M['m10']/M['m00'])
                    #cy = int(M['m01']/M['m00'])
    
                    # 3초 후의 cx 값을 start_cx로 설정
                    if start_cx is None and t_cur - start_time >= 3:
                        start_cx = cx  
    
            cv2.drawContours(crop_img, lines, -1, (0,255,0), 1)
            if len(callbacks) == 0:
                
                ## Predict ##
                if ev_flag and event_cnt-1 in Mission_idx:
                    res = predictor.predict(cap, video_widget,n=10)
                    
                    if event_cnt-1 == 1:
                        tiki.log(f"A AF-{res[0]} EF-{res[1]}")
                        add_callback(callbacks, turn_90, 1.5, 'left')    
                        add_callback(callbacks, tiki.stop, 2) 
                    elif event_cnt-1 == 3:
                        tiki.log(f"B AF-{res[0]} EF-{res[1]}")
                        add_callback(callbacks, turn_90, 1.5, 'right')   
                        add_callback(callbacks, tiki.stop, 2) 
                    elif event_cnt-1 == 5:
                        tiki.log(f"C AF-{res[0]} EF-{res[1]}")
                        add_callback(callbacks, turn_90, 1.5, 'right')       
                        add_callback(callbacks, tiki.stop, 2) 
                    elif event_cnt-1 == 6:
                        tiki.log(f"D AF-{res[0]} EF-{res[1]}")
                        add_callback(callbacks, turn_90, 1.5, 'right')    
                        add_callback(callbacks, tiki.stop, 2) 
                ev_flag = False
                
                if start_cx is not None:
                    adjust_wheel_speed(cx, start_cx)
                
                # Event Detection
                start_x, start_y, end_x, end_y = calculate_center_area(frame)
                center = frame[start_y:end_y, start_x:end_x]
                event_mask = detect_event_color(center, lower_event, upper_event)
                event_detected, event_point = detect_event_in_center(event_mask, start_x, start_y)
                
                if event_detected:
                    event_detection_count += 1
                else:
                    event_detection_count = 0
                
                cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)
        
                if event_point:
                    cv2.circle(frame, event_point, 10, (0, 255, 0), -1)
                    cv2.putText(frame, f"Green: ({event_point[0]}, {event_point[1]})", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Event 
                if cool_time <= 0:
                    cool_flag = 1
                if(event_detection_count >= event_detection_threshold and cool_flag): 
                    cool_time = Event_Handler(callbacks, event_cnt)
                    tiki.stop()
                    ev_flag = True
    
                        
                    event_cnt += 1
                    cool_flag = 0
                if cool_time > 0:
                    cool_time -= 1
          
            frame[roix_s:roix_e, roiy_s:roiy_e] = crop_img
    
            cv2.putText(frame, f"Current Event: {event_cnt}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.putText(frame, f"cx: {cx}s", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    
            cv2.putText(frame, f"Cool Time: {cool_time}s", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.putText(frame, f"Cool Flag: {cool_flag}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.putText(frame, f"EV Flag: {ev_flag}", (10, 170), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    
            _, buffer = cv2.imencode('.jpg', frame)  
            video_widget.value = buffer.tobytes()
            
            # clear_output(wait=True)
            # time.sleep(0.1)  # 프레임 간 간격

if __name__ == '__main__':
    main()

Image(value=b'', format='jpeg')

GST_ARGUS: Creating output stream
CONSUMER: Waiting until producer is connected...
GST_ARGUS: Available Sensor modes :
GST_ARGUS: 3264 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3264 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 120.000005 fps Duration = 8333333 ; Analog Gain range min 1.000000, max 10.625000; Exposur

[ WARN:0@14.348] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1



0: 480x640 1 enemy tank, 193.0ms
Speed: 21.7ms preprocess, 193.0ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 enemy tank, 124.1ms
Speed: 6.9ms preprocess, 124.1ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 enemy tank, 124.9ms
Speed: 6.2ms preprocess, 124.9ms inference, 6.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 enemy tank, 122.8ms
Speed: 6.3ms preprocess, 122.8ms inference, 7.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 enemy tank, 123.7ms
Speed: 5.9ms preprocess, 123.7ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 enemy tank, 122.5ms
Speed: 5.3ms preprocess, 122.5ms inference, 6.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 enemy tank, 125.6ms
Speed: 4.4ms preprocess, 125.6ms inference, 6.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 enemy tank, 122.4ms
Speed: 5.5ms preprocess, 122.4ms inferenc

TypeError: '>' not supported between instances of 'NoneType' and 'int'